# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [32]:
import graphlab
import numpy

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to tor.h.halvorsen@gmail.com and will expire on April 15, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\torhh\AppData\Local\Temp\graphlab_server_1494548264.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [103]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.369559     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.405168     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [104]:
model_all.coefficients.print_rows(num_rows=30)


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

In [105]:
coeff = model_all.coefficients
coeff.sort('value', ascending=False)

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
grade,None,842.068034898,None
sqft_living_sqrt,None,350.060553386,None
sqft_living,None,24.4207209824,None
sqft_above,None,20.0247224171,None
floors_square,None,0.0,None
yr_renovated,None,0.0,None
yr_built,None,0.0,None
sqft_basement,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [28]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [29]:
def polynomial_lasso_regression(data, deg, l1_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=0.,
                                              l1_penalty=l1_penalty,
                                              validation_set=None, 
                                              solver='fista', verbose=False,
                                              max_iterations=3000, convergence_threshold=1e-10)
    return model

In [59]:
def select_l1(training_data, validation_data, features):
    l2_penalty = 0
    l1_penalty_values = numpy.logspace(1, 7, num=13)
    penalty_list=[]
    ret = graphlab.SFrame()
    sa_index = graphlab.SArray(l1_penalty_values, float)
    ret.add_column(sa_index, 'l1_penalty')
    
      
    for l1_penalty in l1_penalty_values:
        model = graphlab.linear_regression.create(training_data, target='price', features=features,
                                                 validation_set=None, 
                                                 l2_penalty=l2_penalty,
                                                 l1_penalty=l1_penalty,
                                                 verbose=False)
         
        # predict on validation set 
        predictions = model.predict(validation_data)
        # compute squared error
        RSS = ((predictions-validation_data['price'])**2).sum()
        penalty_list.append(RSS)
        
    sa_value = graphlab.SArray(penalty_list, float)
    ret.add_column(sa_value, 'RSS')
        
    return(ret)
            
            

In [61]:
pList = select_l1(training, validation, all_features)
pList.print_rows(num_rows=15)

+---------------+-------------------+
|   l1_penalty  |        RSS        |
+---------------+-------------------+
|      10.0     | 6.25766285142e+14 |
| 31.6227766017 | 6.25766285362e+14 |
|     100.0     | 6.25766286058e+14 |
| 316.227766017 | 6.25766288257e+14 |
|     1000.0    | 6.25766295212e+14 |
| 3162.27766017 | 6.25766317206e+14 |
|    10000.0    | 6.25766386761e+14 |
| 31622.7766017 | 6.25766606749e+14 |
|    100000.0   | 6.25767302792e+14 |
| 316227.766017 | 6.25769507644e+14 |
|   1000000.0   | 6.25776517727e+14 |
| 3162277.66017 | 6.25799062845e+14 |
|   10000000.0  | 6.25883719085e+14 |
+---------------+-------------------+
[13 rows x 2 columns]



*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [62]:
pList.sort('RSS', ascending=True)

l1_penalty,RSS
10.0,6.25766285142e+14
31.6227766017,6.25766285362e+14
100.0,6.25766286058e+14
316.227766017,6.25766288257e+14
1000.0,6.25766295212e+14
3162.27766017,6.25766317206e+14
10000.0,6.25766386761e+14
31622.7766017,6.25766606749e+14
100000.0,6.25767302792e+14
316227.766017,6.25769507644e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [102]:
new_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                 validation_set=None, 
                                                 l2_penalty=0,
                                                 l1_penalty=1000000000,
                                                 verbose=False)

new_coeff = new_model.coefficients
sorted_new_coeff = new_coeff.sort('value', ascending=False)
sorted_new_coeff.print_rows(num_rows=30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|    waterfront    |  None | 262245.086458 |  None  |
|   (intercept)    |  None | 79529.8203017 |  None  |
|       view       |  None | 69991.0107955 |  None  |
|    bathrooms     |  None | 23439.0612892 |  None  |
|      floors      |  None | 15907.5888998 |  None  |
|  floors_square   |  None | 8366.26612168 |  None  |
|     bedrooms     |  None | 6348.31466221 |  None  |
|      grade       |  None | 5464.66651869 |  None  |
|    condition     |  None | 4010.27038867 |  None  |
| sqft_living_sqrt |  None | 1038.65224566 |  None  |
|  sqft_basement   |  None | 87.9032707785 |  None  |
|  sqft_lot_sqrt   |  None | 72.3065508156 |  None  |
|    sqft_above    |  None | 40.3786570781 |  None  |
|   sqft_living    |  None | 38.0475711729 |  None  |
| bedrooms_square  |  None | 31.0201101152 |  None  |
|     yr_built     |  None |

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [80]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [81]:
l1_penalty_values = numpy.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [82]:
def select_l1_new(training_data, validation_data, features, l1_penalty_values):
    l2_penalty = 0
    #l1_penalty_values = numpy.logspace(1, 7, num=13)
    penalty_list=[]
    nnz_list=[]
    ret = graphlab.SFrame()
    sa_index = graphlab.SArray(l1_penalty_values, float)
    ret.add_column(sa_index, 'l1_penalty')
    
      
    for l1_penalty in l1_penalty_values:
        model = graphlab.linear_regression.create(training_data, target='price', features=features,
                                                 validation_set=None, 
                                                 l2_penalty=l2_penalty,
                                                 l1_penalty=l1_penalty,
                                                 verbose=False)
         
        # predict on validation set 
        predictions = model.predict(validation_data)
        # compute squared error
        RSS = ((predictions-validation_data['price'])**2).sum()
        penalty_list.append(RSS)
        
        #coeff = model.coefficients
        #coeff.sort('value', ascending=False)
        
        #extract number of non-zero features
        coeff = model['coefficients']['value']
        nnz_list.append( coeff.nnz() )
        
    sa_value = graphlab.SArray(penalty_list, float)
    ret.add_column(sa_value, 'RSS')
    sa_nnz = graphlab.SArray(nnz_list, float)
    ret.add_column(sa_nnz, 'NNZ')
    
       
    return(ret)

In [83]:
new_pList = select_l1_new(training, validation, all_features, l1_penalty_values)
new_pList.print_rows(num_rows=30)

+---------------+-------------------+------+
|   l1_penalty  |        RSS        | NNZ  |
+---------------+-------------------+------+
|  100000000.0  | 6.27492659875e+14 | 18.0 |
|  127427498.57 | 6.28210516771e+14 | 18.0 |
| 162377673.919 | 6.29176689541e+14 | 18.0 |
| 206913808.111 | 6.30650082719e+14 | 18.0 |
| 263665089.873 | 6.32940229287e+14 | 17.0 |
| 335981828.628 |  6.3626814023e+14 | 17.0 |
| 428133239.872 | 6.41261198311e+14 | 17.0 |
| 545559478.117 | 6.48983455376e+14 | 17.0 |
| 695192796.178 | 6.60962217696e+14 | 17.0 |
|  885866790.41 | 6.77261520728e+14 | 16.0 |
| 1128837891.68 | 7.01046815867e+14 | 15.0 |
| 1438449888.29 | 7.37850622829e+14 | 15.0 |
| 1832980710.83 |  7.9616310964e+14 | 13.0 |
| 2335721469.09 | 8.69018172894e+14 | 12.0 |
| 2976351441.63 | 9.66925692362e+14 | 10.0 |
| 3792690190.73 | 1.08186759232e+15 | 6.0  |
| 4832930238.57 | 1.24492736032e+15 | 5.0  |
| 6158482110.66 | 1.38416149024e+15 | 3.0  |
| 7847599703.51 | 1.23079472046e+15 | 1.0  |
| 10000000

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [87]:
l1_penalty_min = -1 
l1_penalty_max = -1
done = False

for i in range(len(new_pList)):
    if(not done):
        if( new_pList[i]['NNZ'] > max_nonzeros ):
            l1_penalty_min = new_pList[i]['l1_penalty']
        if( new_pList[i]['NNZ'] < max_nonzeros ):
            l1_penalty_max = new_pList[i]['l1_penalty']
            done = True

print(l1_penalty_min)
print(l1_penalty_max)


2976351441.63
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [90]:
l1_penalty_values = numpy.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [91]:
new2_pList = select_l1_new(training, validation, all_features, l1_penalty_values)
new2_pList.print_rows(num_rows=30)

+---------------+-------------------+------+
|   l1_penalty  |        RSS        | NNZ  |
+---------------+-------------------+------+
| 2976351441.63 | 9.66925692362e+14 | 10.0 |
| 3019316638.95 | 9.74019450085e+14 | 10.0 |
| 3062281836.27 | 9.81188367942e+14 | 10.0 |
| 3105247033.59 | 9.89328342459e+14 | 10.0 |
| 3148212230.92 | 9.98783211266e+14 | 10.0 |
| 3191177428.24 | 1.00847716702e+15 | 10.0 |
| 3234142625.56 | 1.01829878055e+15 | 10.0 |
| 3277107822.88 | 1.02824799221e+15 | 10.0 |
|  3320073020.2 | 1.03461690923e+15 | 8.0  |
| 3363038217.52 | 1.03855473594e+15 | 8.0  |
| 3406003414.84 | 1.04323723787e+15 | 8.0  |
| 3448968612.16 | 1.04693748875e+15 | 7.0  |
| 3491933809.48 | 1.05114762561e+15 | 7.0  |
| 3534899006.81 | 1.05599273534e+15 | 7.0  |
| 3577864204.13 | 1.06079953176e+15 | 7.0  |
| 3620829401.45 |  1.0657076895e+15 | 6.0  |
| 3663794598.77 | 1.06946433543e+15 | 6.0  |
| 3706759796.09 | 1.07350454959e+15 | 6.0  |
| 3749724993.41 | 1.07763277558e+15 | 6.0  |
| 37926901

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [92]:
for i in range(len(new2_pList)):
    if( new2_pList[i]['NNZ'] == max_nonzeros ):
        print( new2_pList[i] )

{'NNZ': 7.0, 'l1_penalty': 3448968612.1634364, 'RSS': 1046937488751711.1}
{'NNZ': 7.0, 'l1_penalty': 3491933809.4845386, 'RSS': 1051147625612860.9}
{'NNZ': 7.0, 'l1_penalty': 3534899006.8056407, 'RSS': 1055992735342999.1}
{'NNZ': 7.0, 'l1_penalty': 3577864204.126743, 'RSS': 1060799531763287.8}


In [93]:
new_model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.1634364)
coeff = new_model_all.coefficients
coeff.sort('value', ascending=False)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.252702     | 6613330.420898     | 360099.555727 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

name,index,value,stderr
(intercept),None,222253.192544,None
bathrooms,None,15873.9572593,None
grade,None,2899.42026975,None
sqft_living_sqrt,None,690.114773313,None
bedrooms,None,661.722717782,None
sqft_living,None,32.4102214513,None
sqft_above,None,30.0115753022,None
yr_renovated,None,0.0,None
yr_built,None,0.0,None
sqft_basement,None,0.0,None
